In [1]:
import sys
from os.path import join as pj

import sc
from sc.lib import *
import os
import re
import argparse
import logging
import os.path
import time

In [2]:
def get_artifact_path(filename):
    packagedir = sc.__path__[0]
    dirname = pj(os.path.dirname(packagedir), 'share', 'data')
    fullname = os.path.join(dirname, filename)
    return fullname


def get_all_analysis_classes():
    config_file = pkg_resources.resource_stream("sc", 'share/data/artifacts/analysis_classes.csv')
    utf8_reader = codecs.getreader("utf-8")
    c = csv.DictReader(utf8_reader(config_file))
    l = []
    for record in c:
        l.append(record['classes'])
    return l


def choose_classes(acs, data_group_by_class):
    acs = list(map(lambda x: x.replace(".", "_"), acs))
    rm_keys = []
    for dki in data_group_by_class:
        if len(data_group_by_class[dki]) < 30:
            rm_keys.append(dki)
        if dki not in acs:
            rm_keys.append(dki)
    for rk in rm_keys:
        if rk in data_group_by_class:
            del data_group_by_class[rk]
    return data_group_by_class


def dir_check(f):
    if not os.path.isdir(f):
        logger.error("%s is not dir" % f)
        exit(1)

In [3]:
f1 = "./experimental_data/data_folder"

In [4]:
analysis_classes = get_all_analysis_classes()

In [5]:
o1 = get_data_group(f1)

In [6]:
f_budget = "./experimental_data/budget"
if f_budget is not None and f_budget != "":
    dir_check(f_budget)
    o1 = get_data_group(f_budget, o1)

In [7]:
f1_data_group_by_class = choose_classes(analysis_classes, o1)

In [8]:
cns = []
for k,v in f1_data_group_by_class.items():
    class_name = v['con-cbranch-dynamosa-1.2.0']['data'][v['con-cbranch-dynamosa-1.2.0']['classes'][0][1]]['TARGET_CLASS'][0]
    cns.append(class_name)
    print(v.keys())

dict_keys(['con-cbranch-dynamosa-1.2.0', 'con-exce-dynamosa-1.2.0', 'con-output-dynamosa-1.2.0', 'con-methodne-dynamosa-1.2.0', 'con-line-suite-1.2.0', 'con-wm-suite-1.2.0', 'con-branch-suite-1.2.0', 'con-method-suite-1.2.0', 'origin-suite-1.2.0', 'origin-mosa-1.2.0', 'origin-dynamosa-1.2.0', 'sc-dynamosa-sc-release1', 'con-branch-dynamosa-1.2.0', 'con-wm-mosa-1.2.0', 'con-exce-mosa-1.2.0', 'con-output-mosa-1.2.0', 'con-branch-mosa-1.2.0', 'con-wm-dynamosa-1.2.0', 'con-line-dynamosa-1.2.0', 'con-method-dynamosa-1.2.0', 'con-cbranch-suite-1.2.0', 'con-output-suite-1.2.0', 'con-exce-suite-1.2.0', 'con-methodne-suite-1.2.0', 'sc-suite-sc-release1', 'sc-mosa-sc-release1', 'con-cbranch-mosa-1.2.0', 'con-line-mosa-1.2.0', 'con-method-mosa-1.2.0', 'con-methodne-mosa-1.2.0', 'con-cbranch-5-mosa-1.2.0', 'con-exce-5-mosa-1.2.0', 'con-methodne-8-mosa-1.2.0', 'con-output-8-mosa-1.2.0', 'con-exce-8-mosa-1.2.0', 'con-cbranch-8-mosa-1.2.0', 'con-output-5-mosa-1.2.0', 'con-methodne-5-mosa-1.2.0', 'con

In [9]:
cns

['org.apache.hadoop.security.authentication.util.ZKSignerSecretProvider',
 'accessories.plugins.time.JDayChooser']

In [10]:
ad_suite = analysis_data_4_alg(f1_data_group_by_class, 'suite')

In [41]:
def constituent_map_4_budget(algorithm, budget):
    return {
        ("con-branch-%d-%s-1.2.0" % (budget, algorithm)): "BranchCoverageBitString",
        ("con-wm-%d-%s-1.2.0" % (budget, algorithm)): "WeakMutationCoverageBitString",
        ("con-line-%d-%s-1.2.0" % (budget, algorithm)): "LineCoverageBitString",
        ("con-method-%d-%s-1.2.0" % (budget, algorithm)): "MethodCoverageBitString",
        ("con-methodne-%d-%s-1.2.0" % (budget, algorithm)): "MethodNoExceptionCoverageBitString",
        ("con-cbranch-%d-%s-1.2.0" % (budget, algorithm)): "CBranchCoverageBitString",
        ("con-exce-%d-%s-1.2.0" % (budget, algorithm)): "ExceptionCoverageBitString",
        ("con-output-%d-%s-1.2.0" % (budget, algorithm)): "OutputCoverageBitString"
    }
def concat_constituent_one_class_4_budget(algorithm, data_group_by_class, a_class, budget):
    m = constituent_map_4_budget(algorithm, budget)
    g = []
    i = 0
    for k, v in m.items():
        new_value = v.replace('BitString', '')
        if i == 0:
            g.append(data_group_by_class[a_class][k]["data"][a_class])
        else:
            g.append(data_group_by_class[a_class][k]["data"][a_class][v])
            g.append(data_group_by_class[a_class][k]["data"][a_class][new_value])
        if v == 'ExceptionCoverageBitString':
            g.append(data_group_by_class[a_class][k]["data"][a_class]["ExceptionCoverageGoals"])
        size_slice = data_group_by_class[a_class][k]["data"][a_class]["Size"].copy()
        size_slice = size_slice.rename("Constituent" + new_value + "Size")
        g.append(size_slice)
        i = i + 1
    return pd.concat(g, axis=1)
def concat_constituent_all_4_budget(algorithm, data_group_by_class, budget=5):
    for ac in list(data_group_by_class.keys()):
        is_full = True
        ks = constituent_map_4_budget(algorithm, budget)
        for k, _ in ks.items():
            if not k in data_group_by_class[ac]:
                is_full = False
                break
        if not is_full:
            continue
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)] = {}
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["classes"] = \
            data_group_by_class[ac]["origin-%s-1.2.0" % (algorithm)]["classes"]
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["data"] = {}
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["data"][ac] = \
            concat_constituent_one_class_4_budget(algorithm, data_group_by_class, ac, budget)
        data_group_by_class[ac]["constituent-%d-%s" % (budget, algorithm)]["name"] = "constituent-%d-%s" % (budget, algorithm)
    return data_group_by_class

def calc_coverage_4_budget(data_group_by_class, algorithm):
    groups = [re.compile('^sc-(\d+)-%s-sc-release1$' % algorithm), re.compile('^con-(.+)-(\d+)-%s-1.2.0$' % algorithm),
             re.compile('^origin-(\d+)-%s-1.2.0$' % algorithm)]
    for class_name in data_group_by_class.keys():
        for ok, single_data in data_group_by_class[class_name].items():
            match = False
            for rm in groups:
                if rm.match(ok) is not None:
                    match = True
                    break
            if match:
                calcTotal(single_data['data'][class_name])
    return data_group_by_class

In [18]:
f1_data_group_by_class=calc_coverage_4_budget(f1_data_group_by_class, 'suite')

In [65]:
ck = 'org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider'
f1_data_group_by_class[ck]['con-wm-10-suite-1.2.0']['data'][ck]['Size']

0    12
1     3
2     3
3     8
4    15
5     5
Name: Size, dtype: int64

In [42]:
f1_data_group_by_class = concat_constituent_all_4_budget('suite', f1_data_group_by_class, 10)

In [50]:
r = f1_data_group_by_class['org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider']['constituent-10-suite']['data']['org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider']
r.keys()

Index(['TARGET_CLASS', 'search_budget', 'Size', 'Length', 'Generations',
       'BranchCoverageBitString', 'Total_Covered_Goals', 'Total_All_Goals',
       'BranchCoverageGoals', 'BranchCoverageGoalsAll', 'BranchCoverage',
       'ConstituentBranchCoverageSize', 'WeakMutationCoverageBitString',
       'WeakMutationCoverage', 'ConstituentWeakMutationCoverageSize',
       'LineCoverageBitString', 'LineCoverage', 'ConstituentLineCoverageSize',
       'MethodCoverageBitString', 'MethodCoverage',
       'ConstituentMethodCoverageSize', 'MethodNoExceptionCoverageBitString',
       'MethodNoExceptionCoverage', 'ConstituentMethodNoExceptionCoverageSize',
       'CBranchCoverageBitString', 'CBranchCoverage',
       'ConstituentCBranchCoverageSize', 'ExceptionCoverageBitString',
       'ExceptionCoverage', 'ExceptionCoverageGoals',
       'ConstituentExceptionCoverageSize', 'OutputCoverageBitString',
       'OutputCoverage', 'ConstituentOutputCoverageSize'],
      dtype='object')

In [51]:
r = f1_data_group_by_class['org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider']['origin-10-suite-1.2.0']['data']['org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider']
r.keys()

Index(['TARGET_CLASS', 'search_budget', 'Size', 'Length', 'Generations',
       'BranchCoverageBitString', 'WeakMutationCoverageBitString',
       'LineCoverageBitString', 'MethodCoverageBitString',
       'MethodNoExceptionCoverageBitString', 'CBranchCoverageBitString',
       'ExceptionCoverageBitString', 'OutputCoverageBitString',
       'Total_Covered_Goals', 'Total_All_Goals', 'BranchCoverageGoals',
       'BranchCoverageGoalsAll', 'BranchCoverage', 'WeakMutationCoverageGoals',
       'WeakMutationCoverageGoalsAll', 'WeakMutationCoverage',
       'LineCoverageGoals', 'LineCoverageGoalsAll', 'LineCoverage',
       'MethodCoverageGoals', 'MethodCoverageGoalsAll', 'MethodCoverage',
       'MethodNoExceptionCoverageGoals', 'MethodNoExceptionCoverageGoalsAll',
       'MethodNoExceptionCoverage', 'CBranchCoverageGoals',
       'CBranchCoverageGoalsAll', 'CBranchCoverage', 'ExceptionCoverageGoals',
       'ExceptionCoverageGoalsAll', 'ExceptionCoverage', 'OutputCoverageGoals',
       'Ou

In [92]:
def get_mean_4_budget_basic(cname,dgc, name, class_list=None):
    #criterion_map = {
    #    "BC": "ConstituentBranchCoverageSize", "WM": "ConstituentWeakMutationCoverageSize",
    #    "LC": "ConstituentLineCoverageSize", "TMC": "ConstituentMethodCoverageSize",
    #    "NTMC": "ConstituentMethodNoExceptionCoverageSize", "DBC": "ConstituentCBranchCoverageSize",
    #    "EC": "ConstituentExceptionCoverageSize", "OC": "ConstituentOutputCoverageSize"
    #}
    criterion_map = {
        "BC": "BranchCoverage", "WM": "WeakMutationCoverage",
        "LC": "LineCoverage", "TMC": "MethodCoverage",
        "NTMC": "MethodNoExceptionCoverage", "DBC": "CBranchCoverage",
        "EC": "ExceptionCoverageGoals", "OC": "OutputCoverage"
    }
    total_count = 0
    total_sum_dict = {}
    for ck in criterion_map.keys():
        total_sum_dict[ck] = 0
    for class_name, data in dgc.items():
        real_class_name = data[name]['data'][class_name]['TARGET_CLASS'][0]
        if class_list is not None and real_class_name not in class_list:
            continue
        cd = data[name]['data'][class_name]
        sum_dict = cd.sum().to_dict()
        total_count = total_count + len(cd)
        for ck in criterion_map.keys():
            total_sum_dict[ck] = total_sum_dict[ck] + sum_dict[criterion_map[ck]]
    total_mean_dict = {"approach":cname}
    for ck, cv in total_sum_dict.items():
        total_mean_dict[ck] = cv / total_count
    return total_mean_dict

def get_mean_4_budget_constituent(dgc, algorithm, budget, class_list=None):
    return get_mean_4_budget_basic("constituent criterion", dgc, "constituent-%d-%s" % (budget, algorithm), class_list)

def get_mean_4_budget_ss(dgc, algorithm, budget, class_list=None):
     return get_mean_4_budget_basic("smart selection",dgc, "sc-%d-%s-sc-release1" % (budget, algorithm), class_list)

def get_mean_4_budget_origin(dgc, algorithm, budget, class_list=None):
     return get_mean_4_budget_basic("original combination",dgc, "origin-%d-%s-1.2.0" % (budget, algorithm), class_list)
    
def get_mean_size_4_budget(dgc, algorithm, budget, class_list=None):
    criterion_map = {
        "smart selection": ["sc-%d-%s-sc-release1" % (budget, algorithm), "Size"],
         "original combination": ["origin-%d-%s-1.2.0" % (budget, algorithm), "Size"],
        "BC": ["constituent-%d-%s" % (budget, algorithm), "ConstituentBranchCoverageSize"], 
         "WM": ["constituent-%d-%s" % (budget, algorithm), "ConstituentWeakMutationCoverageSize"],
        "LC": ["constituent-%d-%s" % (budget, algorithm), "ConstituentLineCoverageSize"], 
         "TMC": ["constituent-%d-%s" % (budget, algorithm), "ConstituentMethodCoverageSize"],
        "NTMC": ["constituent-%d-%s" % (budget, algorithm), "ConstituentMethodNoExceptionCoverageSize"],
         "DBC": ["constituent-%d-%s" % (budget, algorithm), "ConstituentCBranchCoverageSize"],
        "EC": ["constituent-%d-%s" % (budget, algorithm), "ConstituentExceptionCoverageSize"],
         "OC": ["constituent-%d-%s" % (budget, algorithm), "ConstituentOutputCoverageSize"]}
    total_count = 0
    total_sum_dict = {}
    for ck in criterion_map.keys():
        total_sum_dict[ck] = 0
    for class_name, data in dgc.items():
        r = None
        for ck, cv in criterion_map.items():
            real_class_name = data[cv[0]]['data'][class_name]['TARGET_CLASS'][0]
            if class_list is not None and real_class_name not in class_list:
                continue
            cd = data[cv[0]]['data'][class_name]
            sum_dict = cd.sum().to_dict()
            if r == None:
                r = len(cd)
            total_sum_dict[ck] = total_sum_dict[ck] + sum_dict[cv[1]]
        total_count = total_count + r
    total_mean_dict = {}
    for ck, cv in total_sum_dict.items():
        total_mean_dict[ck] = cv / total_count
    return total_mean_dict

In [44]:
mean_all = get_mean_4_budget_constituent(f1_data_group_by_class, "suite", 10)

In [45]:
mean_all

{'BC': 0.5745192307692307,
 'WM': 0.6468868634769098,
 'LC': 0.671322983014862,
 'TMC': 0.875,
 'NTMC': 0.7744252873563219,
 'DBC': 0.5552884615384616,
 'EC': 16.416666666666668,
 'OC': 0.46811594202898554}

In [95]:
mean_con = get_mean_4_budget_constituent(f1_data_group_by_class, "suite", 10)
mean_ss = get_mean_4_budget_ss(f1_data_group_by_class, "suite", 10)
mean_origin = get_mean_4_budget_origin(f1_data_group_by_class, "suite", 10)


df = pd.DataFrame([mean_ss, mean_origin,mean_con])
df

,approach,BC,WM,LC,TMC,NTMC,DBC,EC,OC
0,smart selection,0.536058,0.643612,0.656740,0.875,0.770115,0.525240,14.166667,0.445652
1,original combination,0.516827,0.641135,0.642781,0.875,0.771552,0.507612,8.750000,0.423188
2,constituent criterion,0.574519,0.646887,0.671323,0.875,0.774425,0.555288,16.416667,0.468116


In [ ]:
get_mean_size_4_budget(f1_data_group_by_class, "suite", 10)

In [106]:
f1_data_group_by_class = concat_constituent_all_4_budget('suite', f1_data_group_by_class, 8)
ad = get_mean_size_4_budget(f1_data_group_by_class, "suite", 8)
aks = list(ad.keys())
avs = []
for ak in aks:
    avs.append(ad[ak])

pd.DataFrame(data={"approach":aks, "Size":avs})

,approach,Size
0,smart selection,56.083333
1,original combination,50.583333
2,BC,33.958333
3,WM,19.208333
4,LC,30.625000
5,TMC,30.125000
6,NTMC,28.583333
7,DBC,40.708333
8,EC,46.750000
9,OC,42.458333


In [81]:
ck = 'org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider'
f1_data_group_by_class[ck]['constituent-10-suite']['data'][ck].mean()

search_budget                               6.000000e+02
Size                                        8.833333e+00
Length                                      1.310000e+02
Generations                                 2.250000e+01
BranchCoverageBitString                              inf
Total_Covered_Goals                         1.283333e+01
Total_All_Goals                             5.200000e+01
BranchCoverageGoals                         1.283333e+01
BranchCoverageGoalsAll                      5.200000e+01
BranchCoverage                              2.467949e-01
ConstituentBranchCoverageSize               8.833333e+00
WeakMutationCoverageBitString                        inf
WeakMutationCoverage                        4.338555e-01
ConstituentWeakMutationCoverageSize         7.666667e+00
LineCoverageBitString                                inf
LineCoverage                                4.522293e-01
ConstituentLineCoverageSize                 7.666667e+00
MethodCoverageBitString        

In [82]:
ck = 'accessories_plugins_time_JDayChooser'
f1_data_group_by_class[ck]['constituent-10-suite']['data'][ck].mean()

search_budget                                6.000000e+02
Size                                         6.133333e+01
Length                                       1.860000e+02
Generations                                  3.091167e+03
BranchCoverageBitString                               inf
Total_Covered_Goals                          1.876667e+02
Total_All_Goals                              2.080000e+02
BranchCoverageGoals                          1.876667e+02
BranchCoverageGoalsAll                       2.080000e+02
BranchCoverage                               9.022436e-01
ConstituentBranchCoverageSize                6.133333e+01
WeakMutationCoverageBitString                         inf
WeakMutationCoverage                         8.599182e-01
ConstituentWeakMutationCoverageSize          3.233333e+01
LineCoverageBitString                                 inf
LineCoverage                                 8.904167e-01
ConstituentLineCoverageSize                  5.250000e+01
MethodCoverage

In [75]:
f1_data_group_by_class.keys()

dict_keys(['org_apache_hadoop_security_authentication_util_ZKSignerSecretProvider', 'accessories_plugins_time_JDayChooser'])